In [1]:
"""
FastF1 client that uses environment configuration
"""

import sys
from pathlib import Path

current = Path.cwd()

indicators = [
    '.git', 'pyproject.toml'
]

for parent in [current] + list(current.parents):
    if any((parent / indicator).exists() for indicator in indicators):
        project_root = parent

sys.path.insert(0, str(project_root))

from typing import List, Optional

import fastf1
import pandas as pd
from fastf1 import Cache

from config.logging import get_logger
from config.settings import fastf1_config

logger = get_logger(__name__)


class FastF1Client:
    """Extending the FastF1Client with data ingestion methods"""

    def __init__(self):
        self.config = fastf1_config

    def get_session(self, year: str, gp: str, session: str):
        """Get a session from the FastF1 API"""

        try:
            # Use FastF1's get_session function
            session_obj = fastf1.get_session(year, gp, session)

            # Configure session based on environment settings
            if hasattr(session_obj, "load"):
                # Load with telemetry based on configuration
                load_telemetry = self.config.enable_telemetry
                load_weather = self.config.enable_weather
                load_race_control_messages = self.config.enable_race_control_messages

                logger.info("Loading session %d %s %s", year, gp, session)
                logger.info("Telemetry: %s, Weather: %s", load_telemetry, load_weather)

                session_obj.load(
                    telemetry=load_telemetry,
                    weather=load_weather,
                    messages=load_race_control_messages,
                )

            return session_obj

        except Exception as e:
            logger.error("Error loading session %d %s %s: %s", year, gp, session, e)
            raise

    def get_season_schedule(self, year: int) -> pd.DataFrame:
        """Get season schedule with configuration"""
        try:
            schedule = fastf1.get_event_schedule(
                year=year, include_testing=self.config.include_testing
            )
            return schedule
        except Exception as e:
            logger.error("Error loading schedule for %d: %s", year, e)
            raise

    def get_multiple_sessions(
        self, year: int, events: List[str], session_types: Optional[List[str]] = None
    ) -> dict:
        """Get multiple sessions efficiently"""

        if session_types is None:
            session_types = self.config.session_types

        sessions = {}

        for event in events:
            sessions[event] = {}
            for session_type in session_types:
                try:
                    session_obj = self.get_session(year, event, session_type)
                    sessions[event][session_type] = session_obj
                    logger.info("Loaded %d %s %s", year, event, session_type)
                except Exception as e:
                    logger.warning(
                        "Could not load %d %s %s: %s", year, event, session_type, e
                    )
                    sessions[event][session_type] = None

        return sessions

    def cache_info(self) -> tuple:
        """Get cache information"""
        return Cache.get_cache_info()

    def clear_cache(self, deep: bool = False):
        """Clear FastF1 cache"""
        Cache.clear_cache(deep=deep)
        logger.info("Cache cleared (deep=%s)", deep)


In [2]:
client = FastF1Client()

In [3]:
schedule_2022 = client.get_season_schedule(2022)
schedule_2022.head()

,RoundNumber,Country,Location,OfficialEventName,EventDate,EventName,EventFormat,Session1,Session1Date,Session1DateUtc,...,Session3,Session3Date,Session3DateUtc,Session4,Session4Date,Session4DateUtc,Session5,Session5Date,Session5DateUtc,F1ApiSupport
2,1,Bahrain,Sakhir,FORMULA 1 GULF AIR BAHRAIN GRAND PRIX 2022,2022-03-20,Bahrain Grand Prix,conventional,Practice 1,2022-03-18 15:00:00+03:00,2022-03-18 12:00:00,...,Practice 3,2022-03-19 15:00:00+03:00,2022-03-19 12:00:00,Qualifying,2022-03-19 18:00:00+03:00,2022-03-19 15:00:00,Race,2022-03-20 18:00:00+03:00,2022-03-20 15:00:00,True
3,2,Saudi Arabia,Jeddah,FORMULA 1 STC SAUDI ARABIAN GRAND PRIX 2022,2022-03-27,Saudi Arabian Grand Prix,conventional,Practice 1,2022-03-25 17:00:00+03:00,2022-03-25 14:00:00,...,Practice 3,2022-03-26 17:00:00+03:00,2022-03-26 14:00:00,Qualifying,2022-03-26 20:00:00+03:00,2022-03-26 17:00:00,Race,2022-03-27 20:00:00+03:00,2022-03-27 17:00:00,True
4,3,Australia,Melbourne,FORMULA 1 HEINEKEN AUSTRALIAN GRAND PRIX 2022,2022-04-10,Australian Grand Prix,conventional,Practice 1,2022-04-08 13:00:00+10:00,2022-04-08 03:00:00,...,Practice 3,2022-04-09 13:00:00+10:00,2022-04-09 03:00:00,Qualifying,2022-04-09 16:00:00+10:00,2022-04-09 06:00:00,Race,2022-04-10 15:00:00+10:00,2022-04-10 05:00:00,True
5,4,Italy,Imola,FORMULA 1 ROLEX GRAN PREMIO DEL MADE IN ITALY ...,2022-04-24,Emilia Romagna Grand Prix,sprint,Practice 1,2022-04-22 13:30:00+02:00,2022-04-22 11:30:00,...,Practice 2,2022-04-23 12:30:00+02:00,2022-04-23 10:30:00,Sprint,2022-04-23 16:30:00+02:00,2022-04-23 14:30:00,Race,2022-04-24 15:00:00+02:00,2022-04-24 13:00:00,True
6,5,United States,Miami,FORMULA 1 CRYPTO.COM MIAMI GRAND PRIX 2022,2022-05-08,Miami Grand Prix,conventional,Practice 1,2022-05-06 14:30:00-04:00,2022-05-06 18:30:00,...,Practice 3,2022-05-07 13:00:00-04:00,2022-05-07 17:00:00,Qualifying,2022-05-07 16:00:00-04:00,2022-05-07 20:00:00,Race,2022-05-08 15:30:00-04:00,2022-05-08 19:30:00,True
